In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/covid_toy.csv')
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [ ]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [ ]:
df.city.value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [ ]:
x_train.shape

(80, 5)

In [ ]:
df.cough.value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

# Ordinary Approach to imputing missing values, encoding nominal categorical values with OneHotEncoder, ordinal categorical values with OrdinalEncoder and Lables with LabelEncoder

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
si = SimpleImputer()

In [ ]:
x_train_fever = si.fit_transform(x_train[['fever']])
x_test_fever = si.transform(x_test[['fever']])

x_test_fever

array([[100.        ],
       [100.        ],
       [103.        ],
       [100.        ],
       [100.        ],
       [100.        ],
       [101.        ],
       [102.        ],
       [ 98.        ],
       [ 98.        ],
       [101.08333333],
       [ 99.        ],
       [101.        ],
       [101.        ],
       [101.08333333],
       [ 98.        ],
       [100.        ],
       [ 98.        ],
       [101.        ],
       [ 98.        ]])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first',sparse_output = False)

x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])
x_test_gender_city = ohe.transform(x_test[['gender','city']])

x_train_gender_city.shape

(80, 4)

In [ ]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])

x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.transform(x_test[['cough']])

x_test_cough

array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.]])

In [ ]:
le = LabelEncoder()
le.fit(y_train)
le.fit(y_test)

LabelEncoder()

In [ ]:
le.classes_


array(['No', 'Yes'], dtype=object)

In [ ]:
y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [ ]:
y_train

array([1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

In [ ]:
x_train_age = x_train.drop(columns = ['gender','age','cough','city'])
x_test_age = x_test.drop(columns = ['gender','age','cough','city'])

In [ ]:
x_train_transformed = np.concatenate((x_train_age,x_train_gender_city,x_train_cough),axis = 1)
x_test_transformed = np.concatenate((x_test_age,x_test_gender_city,x_test_cough),axis = 1)

# Using ColumnTransformer to minimize work time

In [ ]:
from sklearn.compose import ColumnTransformer



In [ ]:
transform = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse_output = False),['gender','city']),
],remainder='passthrough')

In [ ]:
x_train_new = transform.fit_transform(x_train)

In [ ]:
x_train_new

array([[100.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  11.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  56.        ],
       [101.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  51.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  74.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  51.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  82.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  80.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          1.    

In [ ]:
x_test_new = transform.transform(x_test)

In [ ]:
x_test_new


array([[100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  27.        ],
       [103.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  83.        ],
       [100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  19.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  19.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  27.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   8.        ],
       [102.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  49.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.    